# Hospital Readmission Prediction - Neural Networks

This notebook contain the code for Candian Hospital Re-Admission using neural networks.

In [294]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from pprint import pprint

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import optuna

import gc
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Reading the appropriate data and storing it in the dataframe.


In [295]:
df = pd.read_csv("./processed_data.csv")

In [296]:
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]

# for col in X.columns:
#     if (col == 'readmission_id'):
#         print(col)

Code for Neural Network with 4 input layers and one ouput layers.

In [297]:
class ANN(nn.Module):
    def __init__(
        self,
        in_dim: int,
        hidden_dim_1: int,
        hidden_dim_2: int,
        hidden_dim_3: int,
        hidden_dim_4: int,
        n_classes:int = 3,
        dropout: float = 0.3
    ):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(in_features=in_dim, out_features=hidden_dim_1),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim_1),
            nn.Dropout(dropout),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(in_features=hidden_dim_1, out_features=hidden_dim_2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim_2),
            nn.Dropout(dropout),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(in_features=hidden_dim_2, out_features=hidden_dim_3),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim_3),
            nn.Dropout(dropout),
        )
        self.layer4 = nn.Sequential(
            nn.Linear(in_features=hidden_dim_3, out_features=hidden_dim_4),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim_4),
            nn.Dropout(dropout),
        )
        self.output_layer = nn.Linear(in_features=hidden_dim_4, out_features=n_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
            Args:
                x (torch.Tensor): (batch_size, in_dim) the input

            Output:
                (torch.Tensor): (batch_size, n_classes) the output
        """
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.output_layer(x)

        return x

In [298]:
class Data(Dataset):
    def __init__(
        self,
        data
    ):
        n = data.shape[1]
        self.features = torch.tensor(data.iloc[:, 0:n-1].values.astype(np.int64), dtype=torch.float32)
        self.labels = torch.tensor(data.iloc[:, -1].values.astype(np.int64), dtype=torch.int64)
    # def _build(self):
        # scaler = MinMaxScaler(feature_range=())
        # scaler = StandardScaler()

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

    def __len__(self):
        return len(self.features)

Doing the required test,train split with 80:20 ration respectively.

In [299]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0, stratify=Y)
X_train = pd.DataFrame(X_train)
Y_train = pd.DataFrame(Y_train)
X_test = pd.DataFrame(X_test)
Y_test = pd.DataFrame(Y_test)

X_train = pd.concat([X_train, Y_train], axis=1)
X_test = pd.concat([X_test, Y_test], axis=1)

In [300]:
print(X_train.dtypes)

admission_source_id_new_1     bool
admission_source_id_new_2     bool
admission_source_id_new_3     bool
admission_source_id_new_4     bool
admission_source_id_new_5     bool
                             ...  
number_outpatient            int64
patient_id_new               int64
service_utilization          int64
time_in_hospital             int64
readmission_id               int64
Length: 137, dtype: object


Passing the split data into the custom Data class for organization.

In [301]:
train_dataset = Data(data=X_train)
test_dataset = Data(data=X_test)

Setting the batch size.

In [302]:
train_batchsize = 512
val_batchsize = 512

Loading the data into DataLoaded which will then make it useful for loading and handling the pre-processed data into the neural networks.

In [303]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=train_batchsize, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=val_batchsize, shuffle=True)

In [304]:
n_epochs = 5

In [305]:
in_dim = X.shape[1]
in_dim

136

In [306]:
model = ANN(
    in_dim=in_dim,
    hidden_dim_1=in_dim//2,
    hidden_dim_2=in_dim//4,
    hidden_dim_3=in_dim//8,
    hidden_dim_4=3
).to(device)

Using `Adam` optimizer and `Cross Entorpy Loss` for fitting of our model.

In [307]:
lr = 1e-3
optimiser = torch.optim.Adam(model.parameters(), lr=lr)

loss_fn = torch.nn.CrossEntropyLoss()

In [308]:
sanity_check=False

Epoch Code that takes cares of each iteration and uses the Adam optimizer defined above.

In [309]:
def train_epoch(
    model,
    dataloader,
    optimiser
):
    model.train()

    for batch in tqdm(dataloader):
        x, y = batch[0], batch[1]

        output = model(x)
        output = nn.Softmax(dim=-1)(output)
        loss = loss_fn(output, y)

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        if sanity_check:
            break

def validate(
    model,
    dataloader
):
    model.eval()
    total_loss = 0
    predictions = []
    truths = []

    with torch.no_grad():
        for batch in tqdm(dataloader):
            x, y = batch[0], batch[1]

            output = model(x)
            output = nn.Softmax(dim=-1)(output)
            loss = loss_fn(output, y)
            total_loss += loss.detach().cpu().item()/len(dataloader)

            preds = torch.argmax(output, dim=-1)
            predictions.extend(preds.cpu())
            truths.extend(y.cpu())

            if sanity_check:
                break

    acc = accuracy_score(y_true=truths, y_pred=predictions)
    f1 = f1_score(y_true=truths, y_pred=predictions, average='macro')

    return total_loss, acc, f1

Training Model code

In [310]:
def train_model(
    model,
    train_dataloader,
    test_dataloader,
    optimiser
):
    for epoch in range(1, n_epochs+1):
        start_time = time.time()

        print(f"========= EPOCH {epoch} STARTED =========")
        train_epoch(model=model, dataloader=train_dataloader, optimiser=optimiser)

        print(f"========= TRAIN EVALUATION STARTED =========")
        train_val_op = validate(model=model, dataloader=train_dataloader)

        print(f"========= TEST EVALUATION STARTED =========")
        test_val_op = validate(model=model, dataloader=test_dataloader)

        print(f"END OF {epoch} EPOCH")
        print(f"| Time taken: {time.time() - start_time: 7.3f} |")
        print(f"| Train Loss: {train_val_op[0]: 7.3f} | Train acc: {train_val_op[1]: 1.5f} | Train f1: {train_val_op[2]: 1.5f} |")
        print(f"| Test Loss: {test_val_op[0]: 7.3f}  | Test acc: {test_val_op[1]: 1.5f}  | Test f1: {test_val_op[2]: 1.5f}  |")

        if sanity_check:
            break

In [311]:
train_model(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimiser=optimiser
)

========= EPOCH 1 STARTED =========


100%|██████████| 112/112 [00:01<00:00, 70.20it/s]


========= TRAIN EVALUATION STARTED =========


100%|██████████| 112/112 [00:00<00:00, 119.64it/s]


========= TEST EVALUATION STARTED =========


...

========= TRAIN EVALUATION STARTED =========


100%|██████████| 112/112 [00:01<00:00, 100.58it/s]


========= TEST EVALUATION STARTED =========


100%|██████████| 28/28 [00:00<00:00, 131.90it/s]


END OF 5 EPOCH
| Time taken:   9.230 |
| Train Loss:   0.839 | Train acc:  0.71591 | Train f1:  0.48673 |
| Test Loss:   0.845  | Test acc:  0.71105  | Test f1:  0.48229  |


In [312]:
test_data = torch.tensor(X_test.iloc[:, 0:-1].values.astype(np.int64), dtype=torch.float32)
print(model.forward(test_data))

tensor([[-2.1971, -2.7765,  3.0607],
        [ 0.4649,  3.0798, -2.5471],
        [-2.2170, -2.8264,  3.1072],
        ...,
        [ 0.9484,  3.9823, -3.2686],
        [ 0.1518,  2.4330, -1.9150],
        [-2.0864, -2.5203,  2.8366]], grad_fn=<AddmmBackward0>)


Getting the last output and putting it through the `SoftMax` to get the required probabilities.

In [313]:
output_probs = nn.Softmax(dim=-1)(model(test_data))
output_probs

tensor([[5.1649e-03, 2.8935e-03, 9.9194e-01],
        [6.7952e-02, 9.2871e-01, 3.3428e-03],
        [4.8360e-03, 2.6290e-03, 9.9253e-01],
        ...,
        [4.5888e-02, 9.5344e-01, 6.7651e-04],
        [9.1613e-02, 8.9679e-01, 1.1598e-02],
        [7.1908e-03, 4.6597e-03, 9.8815e-01]], grad_fn=<SoftmaxBackward0>)

Getting the max probabilities for the accurate class prediction.

In [314]:
preds = torch.max(output_probs.data, 1)
# print(preds)
vals = preds.indices.numpy()
print(vals)

[2 1 2 ... 1 1 2]


In [315]:
actual = Y_test['readmission_id'].to_numpy()
actual

array([1, 1, 1, ..., 1, 1, 0])

In [316]:
print(accuracy_score(vals, actual))

0.7110471645143178


Using Optuna for hyper-parameter tuning of the trained model.

In [317]:
def objective(trial):
    in_dim = X.shape[1]
    hidden_dim_1 = trial.suggest_int('hidden_dim_1', in_dim // 4, in_dim // 2)
    hidden_dim_2 = trial.suggest_int('hidden_dim_2', in_dim // 8, in_dim // 4)
    hidden_dim_3 = trial.suggest_int('hidden_dim_3', in_dim // 16, in_dim // 8)
    dropout = trial.suggest_float('dropout', 0.2, 0.5)
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)  # Logarithmic scale for lr

    model = ANN(
        in_dim=in_dim,
        hidden_dim_1=hidden_dim_1,
        hidden_dim_2=hidden_dim_2,
        hidden_dim_3=hidden_dim_3,
        hidden_dim_4=3,
        dropout=dropout
    ).to(device)

    optimiser = torch.optim.Adam(model.parameters(), lr=lr)

    train_model(model, train_dataloader, test_dataloader, optimiser)

    _, acc, _ = validate(model=model, dataloader=test_dataloader)
    return acc


In [318]:
def objective1(trial):
    in_dim = X.shape[1]
    num_layers = trial.suggest_int('num_layers', 2, 5)  # Number of layers to optimize
    layer_units = [in_dim] + [trial.suggest_int(f'layer_units_{i}', in_dim // 2**(i+1), in_dim // 2**i) for i in range(num_layers)]
    dropout = trial.suggest_float('dropout', 0.2, 0.5)
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)

    layers = []
    for i in range(num_layers):
        layers.append(nn.Linear(layer_units[i], layer_units[i + 1]))
        layers.append(nn.ReLU())
        layers.append(nn.BatchNorm1d(layer_units[i + 1]))
        layers.append(nn.Dropout(dropout))

    layers.append(nn.Linear(layer_units[-1], 3))  # Assuming 3 output classes
    model = nn.Sequential(*layers).to(device)

    optimiser = torch.optim.Adam(model.parameters(), lr=lr)

    train_model(model, train_dataloader, test_dataloader, optimiser)

    _, acc, _ = validate(model=model, dataloader=test_dataloader)
    return acc


In [319]:
study = optuna.create_study(direction='maximize')
study.optimize(objective1, n_trials=10)  # You can increase n_trials for more optimization

# Get the best hyperparameters found by Optuna
best_params = study.best_params
print("Best hyperparameters:", best_params)

layer_units = [X.shape[1]] + [best_params[f'layer_units_{i}'] for i in range(best_params['num_layers'])]
best_model = nn.Sequential(
    *(nn.Sequential(nn.Linear(layer_units[i], layer_units[i + 1]),
                    nn.ReLU(),
                    nn.BatchNorm1d(layer_units[i + 1]),
                    nn.Dropout(best_params['dropout'])) for i in range(best_params['num_layers'])),
    nn.Linear(layer_units[-1], 3)  # Assuming 3 output classes
).to(device)

best_lr = best_params['lr']
best_optimiser = torch.optim.Adam(best_model.parameters(), lr=best_lr)

train_model(best_model, train_dataloader, test_dataloader, best_optimiser)

# Use the trained model for predictions or further evaluation as needed
test_data = torch.tensor(X_test.iloc[:, 0:-1].values.astype(np.int64), dtype=torch.float32)
output_probs = nn.Softmax(dim=-1)(best_model(test_data))
preds = torch.max(output_probs.data, 1)
vals = preds.indices.numpy()
actual = Y_test['readmission_id'].to_numpy()
print(accuracy_score(vals, actual))

[I 2023-12-15 17:41:16,214] A new study created in memory with name: no-name-79019a2e-93bd-42e5-ab45-d835f48bf10b


========= EPOCH 1 STARTED =========


100%|██████████| 112/112 [00:01<00:00, 69.41it/s]


========= TRAIN EVALUATION STARTED =========


100%|██████████| 112/112 [00:01<00:00, 90.24it/s]


...

========= TRAIN EVALUATION STARTED =========


100%|██████████| 112/112 [00:01<00:00, 102.45it/s]


========= TEST EVALUATION STARTED =========


100%|██████████| 28/28 [00:00<00:00, 123.30it/s]


END OF 5 EPOCH
| Time taken:   9.522 |
| Train Loss:   0.835 | Train acc:  0.71638 | Train f1:  0.49175 |
| Test Loss:   0.840  | Test acc:  0.71112  | Test f1:  0.48656  |
0.7111173498034812


In [320]:
test_df = pd.read_csv("./test_data.csv")
test_df.shape

(30530, 136)

In [321]:
test_data = torch.tensor(test_df.values.astype(np.int64), dtype=torch.float32)
print(best_model.forward(test_data))

tensor([[ 0.5538,  6.6360, -8.0452],
        [-2.0620, -3.6536,  5.9179],
        [ 0.7354,  7.4961, -9.2035],
        ...,
        [ 0.0532,  6.4662, -7.8658],
        [ 0.2993,  6.9499, -8.3833],
        [-2.0339,  3.3378, -1.9788]], grad_fn=<AddmmBackward0>)


In [322]:
output_probs = nn.Softmax(dim=-1)(model(test_data))
preds = torch.max(output_probs.data, 1)
vals = preds.indices.numpy()
vals

array([1, 2, 1, ..., 1, 1, 1])

Getting the output and then storing the ouput in submission2.csv file for submission

In [323]:
df_output = pd.read_csv("./canadian-hospital-re-admittance-challenge/sample_submission.csv")
df_output["readmission_id"] = vals
df_output.to_csv("submission_nn.csv", index=False)

### References
1. List of ICD-9 codes : https://en.wikipedia.org/wiki/List_of_ICD-9_codes
2. Optuna documentation : https://optuna.readthedocs.io/en/stable/
3. Pandas documentation : https://pandas.pydata.org/docs/
4. Numpy documentation : https://numpy.org/doc/1.26/user/index.html
5. Scikit learn documentation : https://scikit-learn.org/stable/
6. Pytorch documentation : https://pytorch.org/docs/stable/index.html